## Advent of Code - Day 4

In [1]:
import tempfile
from contextlib import contextmanager

In [2]:
@contextmanager
def test_file(test_input):
    with tempfile.NamedTemporaryFile('r+') as f:
        f.write(test_input)
        f.seek(0)
        yield f

### Part 1

The idea is to parse the input and sort the records by date and time. Once sorted they can be iterated through in order, incrementing the total time spent asleep by each guard _and_ incrementing each individual minute sleep count. This is $O(n\log n)$ time where $n$ is the total number of records. 

Once this data structure has been created we find the guard whose sleep time is maximal - taking $O(k)$ where $k$ is the total number of guards - and then scan through this guard's array to find the minute with greatest total sleep time ($O(1)$ as there are a fixed number of minutes to scan through!). 

In [3]:
import datetime
import re
from collections import defaultdict

In [4]:
class Record:
    PATTERN = re.compile('^\[(\d\d\d\d)-(\d\d)-(\d\d) (\d\d):(\d\d)]\s*(.*)')
    
    def __init__(self, record_datetime):
        self.record_datetime = record_datetime
        
    def __lt__(self, other):
        return self.record_datetime < other.record_datetime
    
    def __repr__(self):
        params = ', '.join(['%s=%r' % e for e in self.__dict__.items()])
        return self.__class__.__name__ + '(' + params + ')'
    

class BeginShift(Record):
    PATTERN = re.compile('Guard #(\d+) begins shift')
    
    def __init__(self, record_datetime, guard_id):
        super().__init__(record_datetime)
        self.guard_id = guard_id
        

class Awake(Record):
    PATTERN = re.compile('wakes up')


class Asleep(Record):
    PATTERN = re.compile('falls asleep')


def parse_record(record_str):
    # naive regex: use datetime to validate date and time
    match = Record.PATTERN.match(record_str)
    if match is None:
        raise ValueError('invalid `record_str`: %r' % record_str)
        
    groups = match.groups()
    record_datetime = datetime.datetime(year=int(groups[0]),
                                        month=int(groups[1]),
                                        day=int(groups[2]),
                                        hour=int(groups[3]),
                                        minute=int(groups[4]))
    
    match = BeginShift.PATTERN.match(groups[-1])
    if match is not None:
        return BeginShift(record_datetime, int(match.groups()[0]))
    
    for record in [Awake, Asleep]:
        match = record.PATTERN.match(groups[-1])
        if match is not None:
            return record(record_datetime)
                                        
    raise ValueError('invalid `record_str`: %r' % record_str)
    
def parse_records(path):
    with open(path, 'r') as f:
        for record_str in f:
            yield parse_record(record_str)

Try this on the example input (note to self to abstract the test functionality out to a file when there's time!):

In [5]:
test_input = """[1518-11-01 00:00] Guard #10 begins shift
[1518-11-01 00:05] falls asleep
[1518-11-01 00:25] wakes up
[1518-11-01 00:30] falls asleep
[1518-11-01 00:55] wakes up
[1518-11-01 23:58] Guard #99 begins shift
[1518-11-02 00:40] falls asleep
[1518-11-02 00:50] wakes up
[1518-11-03 00:05] Guard #10 begins shift
[1518-11-03 00:24] falls asleep
[1518-11-03 00:29] wakes up
[1518-11-04 00:02] Guard #99 begins shift
[1518-11-04 00:36] falls asleep
[1518-11-04 00:46] wakes up
[1518-11-05 00:03] Guard #99 begins shift
[1518-11-05 00:45] falls asleep
[1518-11-05 00:55] wakes up
"""

with test_file(test_input) as f:
    for record in parse_records(f.name):
        print(record)

BeginShift(record_datetime=datetime.datetime(1518, 11, 1, 0, 0), guard_id=10)
Asleep(record_datetime=datetime.datetime(1518, 11, 1, 0, 5))
Awake(record_datetime=datetime.datetime(1518, 11, 1, 0, 25))
Asleep(record_datetime=datetime.datetime(1518, 11, 1, 0, 30))
Awake(record_datetime=datetime.datetime(1518, 11, 1, 0, 55))
BeginShift(record_datetime=datetime.datetime(1518, 11, 1, 23, 58), guard_id=99)
Asleep(record_datetime=datetime.datetime(1518, 11, 2, 0, 40))
Awake(record_datetime=datetime.datetime(1518, 11, 2, 0, 50))
BeginShift(record_datetime=datetime.datetime(1518, 11, 3, 0, 5), guard_id=10)
Asleep(record_datetime=datetime.datetime(1518, 11, 3, 0, 24))
Awake(record_datetime=datetime.datetime(1518, 11, 3, 0, 29))
BeginShift(record_datetime=datetime.datetime(1518, 11, 4, 0, 2), guard_id=99)
Asleep(record_datetime=datetime.datetime(1518, 11, 4, 0, 36))
Awake(record_datetime=datetime.datetime(1518, 11, 4, 0, 46))
BeginShift(record_datetime=datetime.datetime(1518, 11, 5, 0, 3), guard_i

Looking OK! Now to sort and generate the data structure described above.

In [6]:
def guard_stats(records):
    """Returns a dict of per guard total and per minute sleep counts."""
    stats = defaultdict(lambda: {'total': 0, 'per_min': [0]*60})
    
    cur_guard = None
    asleep = None
    
    for record in sorted(records):
        if isinstance(record, BeginShift):
            cur_guard = record.guard_id
        elif isinstance(record, Awake):
            awake = record.record_datetime.minute
            stats[cur_guard]['total'] += awake - asleep
            for minute in range(asleep, awake):
                stats[cur_guard]['per_min'][minute] += 1
            asleep = None
        elif isinstance(record, Asleep):
            asleep = record.record_datetime.minute
            
    return stats

def tiredest_guard(stats):
    """Returns the tiredest guard's total and per minute sleep counts."""
    tiredest = None
    for guard_id, stat in stats.items():
        if tiredest is None or tiredest[1]['total'] < stat['total']:
            tiredest = (guard_id, stat)
    return tiredest

def argmax(iterable):
    """Computes argmax over the given iterable.""" 
    max_idx = None
    max_val = None
    for i, val in enumerate(iterable):
        if max_val is None or max_val < val:
            max_idx = i
            max_val = val
    return max_idx, max_val

def strategy_1(path):
    """Computes strategy 1 given records from the file at path."""
    guard_id, stats = tiredest_guard(guard_stats(parse_records(path)))
    minute, _ = argmax(stats['per_min'])
    return guard_id * minute

Check that the above works on the test input:

In [7]:
with test_file(test_input) as f:
    print(strategy_1(f.name))

240


Compute using real input:

In [8]:
!ls

day_4.ipynb  input


In [9]:
strategy_1('input')

19874

### Part 2

Part 1 gave us a function which returned the per-minute statistics for all guards in $O(n \log n)$. The idea for part 2 is to iterate of each guard and find the most-frequent minute spent asleep from the corresponding statistics. The maximum minute and frequency will be recorded and used to compute the answer. This is also $O(k)$.

In [10]:
def most_freq_asleep(stats):
    """Returns ID and most freq min of guard who most frequently sleeps."""
    sleepiest_id = None
    sleepiest_min = None
    sleepiest_count = None
    
    for guard_id, stat in stats.items():
        minute, minute_count = argmax(stat['per_min'])
        if sleepiest_count is None or sleepiest_count < minute_count:
            sleepiest_id = guard_id
            sleepiest_min = minute
            sleepiest_count = minute_count
        
    return sleepiest_id, sleepiest_min

def strategy_2(path):
    """Computes strategy 2 given records from the file at path."""
    records = parse_records(path)
    sleepiest_id, sleepiest_minute = most_freq_asleep(guard_stats(records))
    return sleepiest_id * sleepiest_minute

Check that the above works on test input:

In [11]:
with test_file(test_input) as f:
    print(strategy_2(f.name))

4455


Compute using real input:

In [12]:
strategy_2('input')

22687